In [1]:
import pandas as pd
import os
import math

In [22]:
fte = pd.read_csv(os.path.expanduser('~/Downloads/spi_matches.csv'))

In [ ]:
fte.columns

## Put two things online for every league
1. Table
2. Transformed aggregate version w/ one team like cte on sql query

In [3]:
def fte_last_season(path, league_dict):
    df = pd.read_csv(path)
    for key in league_dict:
        q = ''.join(['league_id == ', league_dict[key]])
        league = df.query(q).reset_index()
        num_teams = len(league.loc[:50, 'team1'].unique())
        i = math.floor(league.shape[0]/(num_teams * (num_teams - 1))) - 1
        return league.loc[((num_teams * (num_teams - 1)) * i):,:]

In [23]:
path = os.path.expanduser('~/Downloads/spi_matches.csv')
# league_dict = {'england':'2411'}
league_dict = {'spain':'1869'}
#, 'italy':'1854',
# league_dict = {'germany':'1845'} #, 'france':'1843'}
df = fte_last_season(path, league_dict)

In [24]:
df

,index,season,date,league_id,league,team1,team2,spi1,spi2,prob1,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
1140,23400,2019,2019-08-16,1869,Spanish Primera Division,Athletic Bilbao,Barcelona,73.34,91.18,0.1873,...,35.6,71.4,1.0,0.0,0.53,1.31,0.64,1.60,1.05,0.00
1141,23471,2019,2019-08-17,1869,Spanish Primera Division,Celta Vigo,Real Madrid,67.73,85.18,0.2333,...,24.9,55.7,1.0,3.0,1.63,1.79,1.38,2.33,0.84,2.90
1142,23485,2019,2019-08-17,1869,Spanish Primera Division,Valencia,Real Sociedad,77.77,73.51,0.5380,...,52.3,38.1,1.0,1.0,2.11,1.47,2.09,1.63,1.05,0.84
1143,23489,2019,2019-08-17,1869,Spanish Primera Division,Mallorca,Eibar,63.73,73.21,0.3664,...,38.3,31.5,2.0,1.0,1.56,0.97,1.17,1.66,2.10,1.05
1144,23500,2019,2019-08-17,1869,Spanish Primera Division,Villarreal,Granada,71.24,63.83,0.5492,...,27.4,35.4,4.0,4.0,2.09,1.85,0.98,1.12,4.14,4.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,32826,2019,2020-07-19,1869,Spanish Primera Division,Osasuna,Mallorca,71.00,61.22,0.4310,...,0.0,0.0,2.0,2.0,1.06,0.70,1.14,1.14,2.10,2.10
1516,32837,2019,2020-07-19,1869,Spanish Primera Division,Levante,Getafe,64.52,77.46,0.3175,...,0.0,0.0,1.0,0.0,2.71,1.92,1.27,1.96,1.05,0.00
1517,32838,2019,2020-07-19,1869,Spanish Primera Division,Leganes,Real Madrid,72.53,89.44,0.2534,...,100.0,0.0,2.0,2.0,1.39,1.07,1.32,1.78,2.10,2.10
1518,32839,2019,2020-07-19,1869,Spanish Primera Division,Sevilla FC,Valencia,81.38,66.01,0.4639,...,0.0,0.0,1.0,0.0,0.69,0.29,1.39,0.72,1.05,0.00


In [10]:
def fte_mvp(df):
    home = df.groupby('team1').apply(
        lambda x: (x['score1'] > x['score2']).sum() * 3 + (x['score1'] == x['score2']).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = ['score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1','adj_score2'], 
                        index = 'team1', aggfunc=sum)
    home = home.merge(pt.reset_index(), on = 'team1')
    pt = pd.pivot_table(df, values = 'date', index = 'team1', aggfunc=pd.Series.nunique)
    home = home.merge(pt.reset_index(), on = 'team1')
    cols = {'date':'played', 'team1':'team', 'score1':'scored', 'score2':'conceded', 'xg1':'xg_scored',
            'xg2':'xg_conceded', 'nsxg1':'nsxg_scored', 'nsxg2':'nsxg_conceded', 'adj_score1':'adj_goals_scored',
            'adj_score2':'adj_goals_conceded'}
    home = home.rename(columns = cols)
    away = df.groupby('team2').apply(
        lambda x: (x['score2'] > x['score1']).sum() * 3 + (x['score1'] == x['score2']).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = ['score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1','adj_score2'], 
                        index = 'team2', aggfunc=sum)
    away = away.merge(pt.reset_index(), on = 'team2')
    pt = pd.pivot_table(df, values = 'date', index = 'team2', aggfunc=pd.Series.nunique)
    away = away.merge(pt.reset_index(), on = 'team2')
    cols = {'date':'played', 'team2':'team', 'score2':'scored', 'score1':'conceded', 'xg2':'xg_scored',
            'xg1':'xg_conceded', 'nsxg2':'nsxg_scored', 'nsxg1':'nsxg_conceded', 'adj_score2':'adj_goals_scored',
            'adj_score1':'adj_goals_conceded'}
    away = away.rename(columns = cols)
    t = pd.concat([home, away], ignore_index=True)
    t = t.groupby('team').sum()
    t['goal_difference'] = t['scored'] - t['conceded']
    return t

In [7]:
fte_mvp(df)

,points,adj_goals_scored,adj_goals_conceded,nsxg_scored,nsxg_conceded,scored,conceded,xg_scored,xg_conceded,played,goal_difference,xg_difference,nsxg_difference,adj_goal_difference
team,,,,,,,,,,,,,,
1. FC Union Berlin,41,41.52,58.90,35.36,50.73,41.0,58.0,44.22,52.74,34,-17.0,-8.52,-15.37,-17.38
Bayer Leverkusen,63,61.17,44.29,54.65,36.43,61.0,44.0,62.59,50.37,34,17.0,12.22,18.22,16.88
Bayern Munich,82,99.45,32.04,88.05,29.11,100.0,32.0,92.40,37.50,34,68.0,54.90,58.94,67.41
Borussia Dortmund,69,82.41,42.55,57.37,33.78,84.0,41.0,62.05,41.89,34,43.0,20.16,23.59,39.86
Borussia Monchengladbach,65,65.70,40.26,50.94,48.18,66.0,40.0,71.02,48.06,34,26.0,22.96,2.76,25.44
Eintracht Frankfurt,45,56.87,60.67,51.69,41.03,59.0,60.0,58.94,51.90,34,-1.0,7.04,10.66,-3.80
FC Augsburg,36,45.33,61.36,30.41,52.64,45.0,63.0,43.47,57.67,34,-18.0,-14.20,-22.23,-16.03
FC Cologne,36,50.77,68.47,43.47,48.01,51.0,69.0,49.51,53.88,34,-18.0,-4.37,-4.54,-17.70
Fortuna Düsseldorf,30,37.17,68.40,36.67,50.01,36.0,67.0,36.67,65.92,34,-31.0,-29.25,-13.34,-31.23


In [8]:
def fte_england_tb(df):
    t = fte_mvp(df)
    t = t.sort_values(by=['points', 'goal_difference', 'scored'], ascending = False)
    t['position'] = t.reset_index().index + 1
    return t

In [9]:
fte_england_tb(df)

,adj_goals_conceded,adj_goals_scored,conceded,nsxg_conceded,nsxg_scored,played,points,scored,xg_conceded,xg_scored,goal_difference,xg_difference,nsxg_difference,adj_goal_difference,position
team,,,,,,,,,,,,,,,
Liverpool,21.79,66.60,21.0,26.44,61.85,38,82,66.0,29.78,68.72,45.0,38.94,35.41,44.81,1
Manchester City,30.45,68.39,31.0,21.22,79.44,38,57,68.0,28.66,76.87,37.0,48.21,58.22,37.94,2
Leicester City,29.08,53.87,28.0,36.21,44.03,38,53,58.0,39.10,49.48,30.0,10.38,7.82,24.79,3
Chelsea,40.26,51.77,39.0,26.97,50.95,38,48,51.0,31.47,57.62,12.0,26.15,23.98,11.51,4
Manchester United,30.64,44.59,30.0,33.31,41.68,38,45,44.0,32.97,47.66,14.0,14.69,8.37,13.95,5
Wolverhampton,35.17,41.13,34.0,37.64,34.92,38,43,41.0,32.69,44.64,7.0,11.95,-2.72,5.96,6
Sheffield United,25.95,31.22,25.0,36.60,30.92,38,43,30.0,39.88,37.33,5.0,-2.55,-5.68,5.27,7
Tottenham Hotspur,41.58,48.32,40.0,46.10,36.02,38,41,47.0,46.26,45.70,7.0,-0.56,-10.08,6.74,8
Arsenal,37.04,40.97,36.0,41.37,41.27,38,40,40.0,43.29,39.50,4.0,-3.79,-0.10,3.93,9


In [25]:
def fte_spain_tb(df):
    t = fte_mvp(df)

    if t.duplicated(subset='points', keep = False).sum() > 0:
        t['tb'] = 0
        ties = t[t.duplicated(subset='points', keep = False)]['points'].unique()
        for tie in ties:
            teams = t[t['points'] == tie].index.values
            temp = df.loc[df['team1'].isin(teams) & df['team2'].isin(teams)].dropna()
            if len(temp.index) < (len(teams) * (len(teams) - 1)):
                for team in teams: t.at[team, 'tb'] = 1
            else:
                mini = fte_mvp(temp)
                if mini.duplicated(subset = ['points', 'goal_difference'], keep = False).sum() > 0:
                    mini = pd.merge(mini, t[['goal_difference']], how = 'inner', left_index=True, right_index=True, suffixes=['', '_total'])
                    mini = mini.sort_values(by=['points', 'goal_difference', 'goal_difference_total'], ascending = False)
                else:
                    mini = mini.sort_values(by=['points', 'goal_difference'], ascending = False)

                mini['tb'] = [2 * mini.shape[0]] * len(mini.index) - mini.reset_index().index
                t.update(mini['tb'])
        t = t.sort_values(by=['points', 'tb', 'goal_difference', 'scored'], ascending = False)
        t.drop('tb', axis = 1, inplace=True)
    else:
        t = t.sort_values(by='points', ascending = False)
    t['position'] = t.reset_index().index + 1
    return t

# TROUBLESHOOTING

In [43]:
def fte_spain_tb(df):
    t = fte_mvp(df)

    if t.duplicated(subset='points', keep = False).sum() > 0:
        t['tb'] = 0
        ties = t[t.duplicated(subset='points', keep = False)]['points'].unique()
        for tie in ties:
            teams = t[t['points'] == tie].index.values
            temp = df.loc[df['team1'].isin(teams) & df['team2'].isin(teams)].dropna()
            if len(temp.index) < (len(teams) * (len(teams) - 1)):
                for team in teams: t.at[team, 'tb'] = 1
            else:
                mini = fte_mvp(temp)
                if mini.duplicated(subset = ['points', 'goal_difference'], keep = False).sum() > 0:
                    mini = pd.merge(mini, t[['goal_difference']], how = 'inner', left_index=True, right_index=True, suffixes=['', '_total'])
                    mini = mini.sort_values(by=['points', 'goal_difference', 'goal_difference_total'], ascending = False)
                else:
                    mini = mini.sort_values(by=['points', 'goal_difference'], ascending = False)

                try:
                    mini['tb'] = [2 * mini.shape[0]] * len(mini.index) - mini.reset_index().index
                except:
                    print('tie = ' + str(tie))
                t.update(mini['tb'])
        t = t.sort_values(by=['points', 'tb', 'goal_difference', 'scored'], ascending = False)
        t.drop('tb', axis = 1, inplace=True)
    else:
        t = t.sort_values(by='points', ascending = False)
    t['position'] = t.reset_index().index + 1
    return t

In [42]:
t = fte_mvp(df)
teams = t[t['points'] == 70].index.values
temp = df.loc[df['team1'].isin(teams) & df['team2'].isin(teams)]
mini = fte_mvp(temp)
mini = pd.merge(mini, t[['goal_difference']], how = 'inner', left_index=True, right_index=True, suffixes=['', '_total'])
mini = mini.sort_values(by=['points', 'goal_difference', 'goal_difference_total'], ascending = False)
# print(2 * mini.shape[0] - mini.reset_index().index)
# 2 * mini.shape[0]# - 
[2 * mini.shape[0]] * len(mini.index) - mini.reset_index().index
# mini.shape[0]

Int64Index([4, 3], dtype='int64')

In [20]:
fte_mvp(df)

,points,adj_goals_scored,adj_goals_conceded,nsxg_scored,nsxg_conceded,scored,conceded,xg_scored,xg_conceded,played,goal_difference
team,,,,,,,,,,,
Alavés,35,31.74,50.04,30.61,39.40,32.0,51.0,36.25,51.14,38,-19.0
Athletic Bilbao,48,38.87,32.55,32.41,44.45,39.0,31.0,41.01,48.77,38,8.0
Atletico Madrid,63,47.02,26.79,47.33,34.09,47.0,26.0,57.38,31.57,38,21.0
Barcelona,76,78.84,36.90,84.99,32.62,79.0,36.0,72.14,38.29,38,43.0
Celta Vigo,36,33.64,44.13,38.06,47.23,34.0,44.0,39.71,42.74,38,-10.0
Eibar,36,34.40,51.45,36.95,36.87,34.0,51.0,36.05,48.21,38,-17.0
Espanyol,24,27.51,53.07,35.15,40.33,27.0,55.0,36.79,51.16,38,-28.0
Getafe,53,42.48,34.34,38.12,26.91,43.0,34.0,40.32,39.67,38,9.0
Granada,47,43.25,39.48,30.67,36.52,42.0,40.0,40.73,42.13,38,2.0


In [44]:
fte_spain_tb(df)

,points,adj_goals_scored,adj_goals_conceded,nsxg_scored,nsxg_conceded,scored,conceded,xg_scored,xg_conceded,played,goal_difference,position
team,,,,,,,,,,,,
Real Madrid,87,69.22,26.04,74.91,33.89,70.0,25.0,72.20,35.69,38,45.0,1
Barcelona,82,86.08,39.00,93.65,34.45,86.0,38.0,78.53,40.42,38,48.0,2
Atletico Madrid,70,50.97,27.84,50.63,37.16,51.0,27.0,61.87,34.59,38,24.0,3
Sevilla FC,70,54.65,35.51,52.29,34.79,54.0,34.0,54.62,38.24,38,20.0,4
Villarreal,60,58.74,50.14,44.86,51.39,63.0,49.0,65.97,50.23,38,14.0,5
Real Sociedad,56,57.12,48.59,49.21,44.57,56.0,48.0,54.57,44.30,38,8.0,6
Granada,56,53.23,44.73,35.15,40.45,52.0,45.0,45.58,46.12,38,7.0,7
Getafe,54,42.48,37.24,42.44,29.91,43.0,37.0,44.29,44.41,38,6.0,8
Valencia,53,46.03,50.72,40.63,58.12,46.0,53.0,45.74,58.11,38,-7.0,9


In [8]:
def fte_germany_tb(df):
    # home
    t = fte_mvp(df)
    if t.duplicated(subset=['points', 'goal_difference', 'scored'], keep = False).sum() > 0:
        t['tb'] = 0
        ties = t[t.duplicated(subset='points', keep = False)]['points'].unique()
        for tie in ties:
            teams = t[t['points'] == tie].index.values
            temp = df.loc[df['team1'].isin(teams) & df['team2'].isin(teams)].dropna()
            mini = fte_mvp(temp)
            if mini.duplicated(subset = ['points', 'goal_difference'], keep = False).sum() > 0:
                mini_away = temp.loc[temp['team2'].isin(teams)].dropna()
                mini_away = mvp(mini_away)
                away = mvp(df,date = date, home_team = home_team, away_team = away_team, home_goals = home_goals, away_goals = away_goals)
                mini = pd.merge(mini, mini_away[['team','scored']], how = 'inner', left_index=True,
                                right_on = 'team', suffixes=['', '_away'])
                mini = pd.merge(mini, away[['team','scored']], how = 'inner', on = 'team_name',
                                suffixes=['', '_total_away'])
                mini = mini.sort_values(by=['points', 'goal_difference', 'scored_away', 'scored_total_away'], ascending = False)
            else:
                mini = mini.sort_values(by=['points', 'goal_difference'], ascending = False)
            mini['tb'] = 2 * mini.shape[0] - mini.reset_index().index
            t.update(mini['tb'])
        t = t.sort_values(by=['points', 'tb', 'goal_difference', 'scored'], ascending = False)
        t.drop('tb', axis = 1, inplace=True)
    else:
        t = t.sort_values(by=['points', 'goal_difference', 'scored'], ascending = False)
    t['position'] = t.reset_index().index + 1
    return t

In [13]:
alemania = fte_germany_tb(df)
alemania.to_csv('germany-2019-2020.csv')

In [12]:
alemania

,points,adj_goals_scored,adj_goals_conceded,nsxg_scored,nsxg_conceded,scored,conceded,xg_scored,xg_conceded,played,goal_difference,position
team,,,,,,,,,,,,
Bayern Munich,82,99.45,32.04,88.05,29.11,100.0,32.0,92.40,37.50,34,68.0,1
Borussia Dortmund,69,82.41,42.55,57.37,33.78,84.0,41.0,62.05,41.89,34,43.0,2
RB Leipzig,66,80.01,36.89,62.05,33.11,81.0,37.0,71.91,39.46,34,44.0,3
Borussia Monchengladbach,65,65.70,40.26,50.94,48.18,66.0,40.0,71.02,48.06,34,26.0,4
Bayer Leverkusen,63,61.17,44.29,54.65,36.43,61.0,44.0,62.59,50.37,34,17.0,5
TSG Hoffenheim,52,52.88,53.26,46.16,53.27,53.0,53.0,58.34,59.89,34,0.0,6
VfL Wolfsburg,49,48.69,46.01,44.43,40.07,48.0,46.0,54.25,41.56,34,2.0,7
SC Freiburg,48,48.10,47.05,41.27,59.20,48.0,47.0,46.65,64.51,34,1.0,8
Eintracht Frankfurt,45,56.87,60.67,51.69,41.03,59.0,60.0,58.94,51.90,34,-1.0,9
